In [3]:
import numpy as np
import pandas as pd
from ContentTFIDF import ContentTFIDF

path = pd.read_csv('./new.csv', encoding = 'utf-8', sep='\t')
ctfidf = ContentTFIDF(path)
tfidf = ctfidf.calculateTFIDF()
print(tfidf.head())

ParserError: Error tokenizing data. C error: Expected 22 fields in line 3, saw 28
